<a href="https://colab.research.google.com/github/chuckfinca/arithmephi/blob/main/arithmephi_inference_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==2.1.0
!pip install pyarrow==14.0.2
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
# peft is required
!pip install einops accelerate peft

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vlxsz8y2
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vlxsz8y2
  Resolved https://github.com/huggingface/transformers to commit 0290ec19c901adc0f1230ebdccad11c40af026f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661022 sha256=39493db8d9cf053f3f82e8bf76faa5346e2408578f875cac9b4336677bf8ce6c
  Stored in directory: /tmp/pip-ephem-wheel-cache-2zex2i0r/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
     ━━━━

In [4]:
!cd arithmephi && touch handler.py

In [2]:
from typing import Dict, List, Any
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
class EndpointHandler():
    def __init__(self, path=""):
        torch.set_default_device("cuda")
        self.model = AutoModelForCausalLM.from_pretrained("chuckfinca/arithmephi", torch_dtype="auto", trust_remote_code=True, device_map = 'cuda')
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

    def __call__(self, data: Dict[str, Any]) -> List[Dict[str, Any]]:
        input = data.get("inputs", data)
        inputs = self.tokenizer(input, return_tensors="pt", return_attention_mask=False).to('cuda')
        outputs = self.model.generate(**inputs, max_length=len(inputs[0]) + 8, pad_token_id=self.tokenizer.eos_token_id)
        text = self.tokenizer.batch_decode(outputs)[0]
        return text

In [9]:
# init handler
my_handler = EndpointHandler(path="chuckfinca/arithmephi")
# my_handler = EndpointHandler(path="microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# prepare sample payload
request = {
  "inputs": "Long ago in a galaxy far away"
}

# test the handler
 print(my_handler(request))

Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
Long ago in a galaxy far away, there was a planet called Earth.
697 ms ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
